### Script to Create Choropleth Maps
<font size="3"><p>The purpose of this script is to create choropleth maps of the parking costs data. Current script calculates 'monthly/hourly' and 'daily/hourly' ratios, and adds MAZ geometry before generating these maps. The function to generate these choropleth maps has several settings (some set to default that could be easily changed). This function by default creates a "stamentoner style" map with a linear colorscale in "San Diego region". At the time being this function can cap the colorscale (both upper and lower), and change a colorscale to a step version.</p></font>

<font size="3">*Last Modified By:* Rohan Sirupa</font>

<font size="3">*Last Modified Date:* May 15th, 2023</font>

In [45]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime

pd.set_option("display.max_columns", 100)
start_time = datetime.datetime.now()

In [46]:
# Define Data Paths
### Accessibility files
consolidated_costs_loc = "output/consolidated_costs.csv"
imputed_costs_loc = "output/imputed_costs.csv"

### Shape files
taz_loc = "data/taz15/taz15.shp"
maz_loc = "data/mgra15/mgra15.shp"

### Location to save maps
maps_loc = "output/colormaps"

In [47]:
### Read files
consolidated_df = pd.read_csv(consolidated_costs_loc)
imputed_df = pd.read_csv(imputed_costs_loc)
# taz_gdf = gpd.read_file(taz_loc)
maz_gdf = gpd.read_file(maz_loc)

### Change shp CRS to 4326 (Latitude and Longitude System)
maz_gdf = maz_gdf.to_crs(4326)
# taz_gdf = taz_gdf.to_crs(4326)

### Create a crosswalk between TAZ and MAZ
# maz_to_taz_xwalk = maz_gdf.set_index('MGRA')['TAZ'].to_dict()

### Use crosswalk to add TAZ to dataframe
# consolidated_df['TAZ'] = consolidated_df['mgra'].map(maz_to_taz_xwalk)
# imputed_df['TAZ'] = imputed_df['mgra'].map(maz_to_taz_xwalk)

In [48]:
### Sample custom colormap
custom_colorscale = folium.LinearColormap(['green','yellow','red'], vmin=3., vmax=10.)
custom_colorscale

In [49]:
### Sample step custom colormap
custom_colorscale.to_step(
    n=7,
    data=np.arange(3, 10+1, (10-3)/7),  # create n equal steps between vmin and vmax of colormap
    method='quantiles',
    round_method='int'
)

In [50]:
### Function to calculate metric and add geometry
def calculate_ratio_add_geometry(
        df, 
        index='mgra', 
        ref_gdf=maz_gdf[['MGRA', 'geometry']], 
        ref_gdf_col='MGRA',):
    
    ### Fill na with zeros
    df[['hourly', 'daily', 'monthly']] = df[['hourly', 'daily', 'monthly']]#.fillna(0)

    ### Calculate cost ratios
    df['daily_hourly'] = df['daily']/df['hourly']
    df['monthly_hourly'] = df['monthly']/df['hourly']
    
    ### Add geometry from TAZ shp file
    df = pd.merge(
        df, 
        ref_gdf, 
        how='left', 
        left_on=index, right_on=ref_gdf_col, 
        suffixes=('', '_x')
    ).drop([ref_gdf_col], axis=1)
    gdf = gpd.GeoDataFrame(df)
    gdf = gdf[~gdf.geometry.isna()]

    return gdf

consolidated_gdf = calculate_ratio_add_geometry(df=consolidated_df)
imputed_gdf = calculate_ratio_add_geometry(df=imputed_df)

In [51]:
### Function to create colormap
def generate_choropleth_map(
        gdf, # gpd.GeoDataFrame that you want to display
        output_map_dir, # directory to save the HTML file
        filename, # name of the HTML file
        colorscale_caption, # caption to be displayed below colorscale
        fill_column='TAZ_Mean', # column or values used to create the colormap
        tooltip_list=['TAZ', 'TAZ_Min', 'TAZ_Max', 'TAZ_Mean', 'Global_Min', 'Global_Max', 'Global_Mean'], # statistics to display in tooltip
        step_colormap=False, # option to change to a step colormap instead of the default linear colormap
        num_color_steps=7, # preferred number of steps in the colormap
        cap_upper_limit=None, # upper limit when capping the fill_column
        cap_lower_limit=None, # lower limit when capping the fill_column
        coordinates=[33.043, -116.756], # initial co-ordinates of map, SANDAG
        tile_layer='stamentoner', # base folium layer
        tile_name='stamentoner', # title for base layer
        fillOpacity=0.75, # opacity of the choropleth layer over base map
        save_map=True): # option to save map as an HTML
    
    ### Create a base Folium map for region specified in coordinates
    map = folium.Map(location=coordinates, zoom_start=10, tiles=None, overlay=False) 

    ### Adding tiles as a separate layer allows for more control and more customization
    folium.TileLayer(tile_layer, name=tile_name, control=False, opacity=0.25).add_to(map)

    ### Cap lower and upper limits of the fill_column
    gdf[fill_column + '_capped'] = gdf[fill_column].clip(lower=cap_lower_limit, upper=cap_upper_limit)

    ### Create a custom color scale
    colormap = folium.LinearColormap(
        ['green','yellow','red'], 
        vmin=np.floor(gdf[fill_column + '_capped'].min()), 
        vmax=np.ceil(gdf[fill_column + '_capped'].max()),
        caption=colorscale_caption
        )
    
    ### Change linear colormap to step colormap
    if step_colormap:
        colormap = colormap.to_step(
            n=num_color_steps,
            data=np.percentile(gdf[fill_column + '_capped'], np.arange(0, 100+1, (100)/num_color_steps)),
            method='quantiles',
            round_method='int'
        )

    ### Add the colormap as a legend
    colormap.add_to(map)
    
    ### Missing data color as gray
    def get_color(feature):
        value = feature['properties'][fill_column]
        if value == 0 or value is None:
            return '#8c8c8c' # MISSING -> gray
        else:
            return colormap(feature['properties'][fill_column + '_capped'])
    

    ### Creating a function that would call on these values
    style_function = lambda x: {"weight": 0, 
                                'color': 'black',
                                # 'fillColor': colormap(x['properties'][fill_column + '_capped']), 
                                'fillColor': get_color(x), 
                                'fillOpacity': fillOpacity}
    highlight_function = lambda x: {'weight': 3,
                                    'fillColor': get_color(x),
                                    # 'fillColor': colormap(x['properties'][fill_column + '_capped'])
                                    }

    ### Add the colormap
    folium.features.GeoJson(
        data=gdf,
        style_function=style_function,
        tooltip=folium.features.GeoJsonTooltip(tooltip_list),
        highlight_function=highlight_function
    ).add_to(map) 

    ### Export map as HTML
    if save_map:
        map.save(output_map_dir + "/" + filename + ".html")

    ### Return the map so other changes can be made later as needed
    return map

### Writing and saving maps

Create choropleth maps of hourly, daily, and monthly parking costs

In [52]:
### Create colormaps for consolidated data
file_tag = 'consolidated'
gdf = calculate_ratio_add_geometry(consolidated_df)

for column_name in ['hourly', 'daily', 'monthly']:
    generate_choropleth_map(
        gdf=gdf.copy(), 
        output_map_dir=maps_loc, 
        filename=file_tag+'_'+column_name, 
        colorscale_caption=column_name,
        fill_column=column_name,
        tooltip_list=['mgra', 'hourly', 'daily', 'monthly', 'daily_hourly', 'monthly_hourly'])
    
for column_name in ['daily_hourly', 'monthly_hourly']:
    generate_choropleth_map(
        gdf=gdf[~gdf[column_name].isna()].copy(), 
        output_map_dir=maps_loc, 
        filename=file_tag+'_'+column_name, 
        colorscale_caption=column_name,
        cap_upper_limit=max(gdf.loc[gdf[column_name]!=np.inf, column_name]),
        fill_column=column_name,
        tooltip_list=['mgra', 'hourly', 'daily', 'monthly', 'daily_hourly', 'monthly_hourly'])

C:\Users\nick.fournier\AppData\Local\Temp\ipykernel_13396\528178620.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf[fill_column + '_capped'] = gdf[fill_column].clip(lower=cap_lower_limit, upper=cap_upper_limit)
C:\Users\nick.fournier\AppData\Local\Temp\ipykernel_13396\528178620.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf[fill_column + '_capped'] = gdf[fill_column].clip(lower=cap_lower_limit, upper=cap_upper_limit)
C:\Users\nick.fournier\AppData\Local\Temp\ipykernel_13396\528178620.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [53]:
### Create colormaps for imputed data
file_tag = 'imputed'
cols = ['hourly', 'daily', 'monthly']
gdf = imputed_df.drop(columns=cols).rename(columns={k + '_imputed': k for k in ['hourly', 'daily', 'monthly']})
del cols

gdf = calculate_ratio_add_geometry(gdf)

for column_name in ['hourly', 'daily', 'monthly']:
    generate_choropleth_map(
        gdf=gdf.copy(), 
        output_map_dir=maps_loc, 
        filename=file_tag+'_'+column_name, 
        colorscale_caption=column_name,
        fill_column=column_name,
        tooltip_list=['mgra', 'hourly', 'daily', 'monthly', 'daily_hourly', 'monthly_hourly'])
    
for column_name in ['daily_hourly', 'monthly_hourly']:
    generate_choropleth_map(
        gdf=gdf[~gdf[column_name].isna()].copy(), 
        output_map_dir=maps_loc, 
        filename=file_tag+'_'+column_name, 
        colorscale_caption=column_name,
        cap_upper_limit=max(gdf.loc[gdf[column_name]!=np.inf, column_name]),
        fill_column=column_name,
        tooltip_list=['mgra', 'hourly', 'daily', 'monthly', 'daily_hourly', 'monthly_hourly'])

In [54]:
end_time = datetime.datetime.now()
print("Start Time:", start_time)
print("End Time:", end_time)
print("Run Time:", round(end_time.timestamp()-start_time.timestamp(), 3), "sec")

Start Time: 2023-05-16 11:43:58.331575
End Time: 2023-05-16 11:44:17.866414
Run Time: 19.535 sec
